In [1]:
import warnings
import tensorflow as tf
import tensorflow_datasets as tfds
import matplot
print(tf.__version__)

2.2.0


In [2]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [3]:
IMG_SIZE = 180 # All images will be resized to 160x160

def format_example(image, label):
    image = tf.cast(image, tf.float32)  # image=float(image)같은 타입캐스팅의  텐서플로우 버전입니다.
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [4]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [5]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [6]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [7]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model VGG16
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                         include_top=False,
                                         weights='imagenet')

In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [9]:
dense_layer = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')

In [10]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  dense_layer,
  prediction_layer
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 5, 5, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 14,979,909
Trainable params: 14,979,909
Non-trainable params: 0
_________________________________________________________________


In [12]:
base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])


In [13]:
EPOCHS = 20   # 이번에는 이전보다 훨씬 빠르게 수렴되므로 5Epoch이면 충분합니다.

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/20
92/92 [==============================] - 86s 930ms/step - loss: 0.7716 - accuracy: 0.7197 - val_loss: 0.5238 - val_accuracy: 0.8120
Epoch 2/20
92/92 [==============================] - 24s 263ms/step - loss: 0.4100 - accuracy: 0.8522 - val_loss: 0.4031 - val_accuracy: 0.8474
Epoch 3/20
92/92 [==============================] - 24s 264ms/step - loss: 0.3133 - accuracy: 0.8835 - val_loss: 0.3857 - val_accuracy: 0.8610
Epoch 4/20
92/92 [==============================] - 24s 265ms/step - loss: 0.2384 - accuracy: 0.9203 - val_loss: 0.3609 - val_accuracy: 0.8638
Epoch 5/20
92/92 [==============================] - 24s 257ms/step - loss: 0.1846 - accuracy: 0.9370 - val_loss: 0.3459 - val_accuracy: 0.8747
Epoch 6/20
92/92 [==============================] - 24s 256ms/step - loss: 0.1407 - accuracy: 0.9520 - val_loss: 0.4791 - val_accuracy: 0.8583
Epoch 7/20
92/92 [==============================] - 24s 264ms/step - loss: 0.1068 - accuracy: 0.9656 - val_loss: 0.3516 - val_accuracy: 0.8774

In [14]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
test_steps = 10
loss0, accuracy0 = model.evaluate(test_batches, steps = test_steps)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))